# N-Grams and Maximum Likelihood Estimation

## Quick Review
As you've seen in lecture and/or in the reading, n-gram language models are structured around taking a direct, empirical approach to solving the language modeling task: We compute $p(w \mid C)$ by estimating $p(w, C)$ and $p(C)$ by counting occurances in a large corpus of text. 

However, as you saw, the context, $C$, can be really long, and we probably don't have enough data (or, at least, data that you can process on your computer) to get a good estimate of $p(w, C)$ and $p(C)$. N-grams models handle this by making a **Markov Assumption**: We assume that we only need a fixed amount of prior context to predict the next word. How much prior context? An $n$-gram model assumes a context of $n-1$ words!

That is, we assume $p(w \mid C) \approx p(w \mid w_{-1}, \dots w_{-n+1})$. If we label our random variables such that a sentence is a series of random variables $W_1, \dots$, we get the much nicer form $p(w_k \mid w_1 \dots w_{k-1}) \approx p(w_k \mid w_{k-n} \dots w_{k-1})$. More casually, $p(\text{next word} \mid \text{prior context}) \approx p(\text{next word} \mid \text{past $n$ words})$.

## This Assignment

Our learning goals for this assignment are to...

    1. Understand how an n-gram language model is trained and used.
    2. Implement an n-gram language model and training using Maximum Likelihood Estimation

To start, let's load in some data: the text of Jane Austin's Emma, via Project Gutenberg. 

In [3]:
import pickle
import numpy as np

with open("austen-emma.txt") as train_f:
    train = train_f.read().lower().split()

print(train[:10])

['[', 'emma', 'by', 'jane', 'austen', '1816', ']', '<eos>', 'volume', 'i']


You can see that we do some light pre-processing --- we lowercase all of the text. You should also see that the corpus you've been given has been annotated with `\<eos>` markers. Like the `\<s>` and `\<\s>` tokens you saw in the reading, these tokens serve to mark sentence boundaries. In classical NLP, these markers are often used so that separate sentences are distinct from eachother. In modern, neural NLP, you'll find that such careful preprocessing has largely been abandoned. 

Before we get to start dealing with n-gram models at all, we need to do some addition, n-gram specific pre-processing on our data. The first task is to get counts for n-grams, and before we get there, it's prudent to be able to split our data into overlapping $n$-word sequences. 

For example, "This is a sentence . \<eos>" would turn into the list of bigrams, each of which is represented by a tuple \[("this", "is"), ("is", "a"), ("a", "sentence"), ("sentence", "."), (".". "\<eos>")]. 

In [4]:
from typing import Sequence, Iterable

def get_ngrams(words : Sequence[str], n : int) -> Iterable[tuple[str]]:
    # TODO: Complete this implementation
    return []

print(list(get_ngrams("this is a sentence . <eos>".split(), 2)))

[]


Now we could go ahead and start counting ngrams now, but there's one thing to observe: We don't ever get to estimate the probability of the first n-1 words of the text. That is, we never use the fact that "this" is the first word in the sentence because there is no bigram where "this" is the second word! 

As you saw in the reading, this calls for *padding* the text. That is, adding a "boundary" of several separator tokens so that ngram models can estimate the probability of beginning a sentence/paragraph/text. It was convention in what I'll call "classical" language modeling to separate sentences from each other with a sequence of padding tokens, though in modern NLP such conventions have fallen to the wayside due to the scale of data involved.

For this assignment, let's pad *sentence by sentence*

First, ask yourself:

1. For an $n$-gram model, how many padding tokens should be between each sentence? Remember that we want to have an ngram correspond to the likelihood of the first word of the sentence being the first word of the sentence!
2. How are sentence boundaries currently marked? How can you use those to create the proper amount of padding tokens?

Fill in an algorithm to pad a text in the Gutenberg corpus format of Austen's Emma above. Use `<eos>` as a padding token.

In [5]:
def pad(words : Sequence[str]) -> Sequence[str]:
    # TODO: Complete this implementation
    return []

Now you have code to roughly preprocess data for language modeling!

Of course, you can preprocess *more* if you wish, typically in the interest of avoiding rare or unseen tokens in your vocabulary. Sometimes folks replace all numbers with a single token `<num>` so the model just has to predict a number comes next. Other times, you take all of the super infrequent words and replace them with `<unk>` (unknown) (or `<oov>`/out-of-vocabulary) tokens, and preprocess any data after training to have `<unk>`s instead of any out-of-vocab words. For now, we can hold off on those tricks.  

Our next step is to *train* our model. Here, that notion refers to picking a training corpus (our preprocessed Emma) and learning some *parameters* (that is, numbers in our model we don't know a priori) using that corpus. For an n-gram model, those parameters are our counts of each ngram!

Complete the function below that does this, and test that it works using a few examples.

In [6]:
from typing import Mapping
def get_counts(train : Sequence[str], n : int) -> Mapping[str, int]:
    counts = {}
    # TODO: count the occurances of each ngram in train
    return counts

And now we're finally ready to build an ngram model! Below is a template for an mle_ngram_lm (that is, an MLE-trained ngram LM). Complete the two sections marked TODO to complete the implementation.

In [51]:
class mle_ngram_lm:
    def __init__(self, train : Sequence[str], n : int) -> None:
        # TODO: train the model here, declaring appropriate instance variables (i.e., model parameters!)
        # Consider what you need to compute logprob(w, c)!
        pass

    def logprob(self, w : str, c : Sequence[str]) -> float:
        assert (len(c) + 1) == self.n
        # TODO: compute p(w | c) using those instance variables
        return 0.0


When the above code is complete, test the implementation on a small example:

1. Write a few sentences.
2. Preprocess those sentences.
3. **By hand** compute the frequencies of a few unigrams/bigrams/trigrams. Make sure there are a few bigrams/trigrams with probability strictly between 0 and 1!
4. Train your model on those sentences and confirm your code's output and your by-hand calculations match!
5. Modify your sentences if you've spotted a tricky edge case you can construct, or modify your code (or, perhaps, your by-hand calculations!) if there was a mismatch in (4)!

Once you've done this to your satisfaction (your convinced your code is right!), train the model on the full text of Emma in the cell below. Be sure to apply the full pre-processing pipeline.

**Bonus:** Given extra time, consult the reading for a definition of *perplexity*, a measure often used to determine how good a language model is. In the cell below, complete an implementation of perplexity and use it to measure the perplexity of your n-gram model over it's training data. 

Consider the following questions
1. For a given $n$, can any $n$-gram model have a lower (i.e., better!) perplexity over the training set than the one you trained?
2. How about an $m$-gram model for any $m$?

In [7]:
def ppl(eval_data : Iterable[str], model) -> float:
    # TODO compute perplexity!
    return 0.0